📘 Example: Creating a Hurricane Event in FloodAdapt

This notebook demonstrates how to create a hurricane event using FloodAdapt. Hurricane events are valuable for controlled testing, sensitivity analysis, and understanding the behavior of flood models under simplified or hypothetical scenarios. 

A FloodAdapt Event consists of 2 things:
- a TimeFrame describing the start and end time of the hazard simulation(s)
- a collection of forcings to be applied to the hazard model(s)

In this example, we construct a full hurricane event with water level, rainfall, wind, and river discharge forcings, and then save it to a FloodAdapt database.

⏱️ 1. Setup and Imports

We begin by importing the required classes and modules for constructing hurricane forcings and managing event data within the flood_adapt framework.

In [ ]:
import flood_adapt.objects.forcing as f

from pathlib import Path
from datetime import datetime

from flood_adapt.objects import HurricaneEvent, TimeFrame
from flood_adapt.objects.events.hurricane import TranslationModel
from flood_adapt import unit_system as us
from flood_adapt import FloodAdapt
from flood_adapt.config.sfincs import RiverModel

🗓️ 2. Define the Simulation Time Frame

We specify a one-day time frame for the hurricane event, from January 1 to January 2, 2025.

In [ ]:
# Create an time frame for the simulation
start_time = datetime(year=2025, month=1, day=1)
end_time = datetime(year=2025, month=1, day=2)
time_frame = TimeFrame(start_time=start_time, end_time=end_time)

🌊 3. Define Water Level Forcing

Water levels for Hurricane Events are computed by taking the Hurricane Track, and generating a pressure and wind field along its track.

These fields are then used as forcing inputs to the offshore simulation, which generates the storm surge to be used for the overland simulation.

In [ ]:
water_levels = f.WaterlevelModel()

🌧️ 3. Obtain a hurricane track

The FloodAdapt database can store a cyclone track database as well. Depending on your setup and configuration, this functionality will be available.


In [ ]:
# Get the cyclone database
fa = FloodAdapt(database_path=Path("../../../../Database/charleston_test")) # TODO
cyclone_db = fa.database.static.get_cyclone_track_database()
ian_index = cyclone_db.list_names().index("IAN")

# Not all cyclone tracks have names, in addition to duplicate names existing, so it is better to use the index
track = fa.get_cyclone_track_by_index(index=ian_index) 
track_file = Path("../../_data/IAN.cyc")
track.write_track(filename=track_file, fmt="ddb_cyc")

# Translate the cyclone track from what is defined in the file
translation = TranslationModel(
    eastwest_translation=us.UnitfulLength(value=3000, units=us.UnitTypesLength.meters),
    northsouth_translation=us.UnitfulLength(value=5000, units=us.UnitTypesLength.meters),
)

🌧️ 4. Create Track forcings

Given a Hurricane track, it is quite easy to create hurricane events.

In [ ]:
# We want to include the rainfall and wind from the hurricane track
rainfall = f.RainfallTrack(path=track_file)
wind = f.WindTrack(path=track_file)

🏞️ 5. Define River Discharge Forcing

Discharge is defined for two pre-configured rivers. These rivers must be registered in the hazard model configuration beforehand.

In [ ]:
# Add discharge
# The available rivers are defined in the hazard model when creating the database.
# You cannot add new rivers to the model in an event, you can only set the discharge of each given river.
river1 = RiverModel(
    name="River1",
    x_coordinate=1,
    y_coordinate=1,
    mean_discharge=us.UnitfulDischarge(value=100, units=us.UnitTypesDischarge.cms), 
    # Mean discharge is used create default discharge values for the river, you can still overwrite it later.
)
river2 = RiverModel(
    name="River2",
    x_coordinate=2,
    y_coordinate=2,
    mean_discharge=us.UnitfulDischarge(value=200, units=us.UnitTypesDischarge.cms),
    # Mean discharge is used create default discharge values for the river, you can still overwrite it later.
)

discharge1 = f.DischargeConstant(
    river=river1,
    discharge=us.UnitfulDischarge(value=100, units=us.UnitTypesDischarge.cms)
)
discharge2 = f.DischargeConstant(
    river=river2,
    discharge=us.UnitfulDischarge(value=200, units=us.UnitTypesDischarge.cms)
)

# Inspect
df1 = discharge1.to_dataframe(time_frame=time_frame)
df2 = discharge2.to_dataframe(time_frame=time_frame)
df_combined = df1.join(df2)
df_combined.plot(title="Constant Discharge River", xlabel="Time", ylabel="Discharge (cms)", legend=True, figsize=(5, 2))

🧩 6. Combine Forcings and Create hurricane Event

All defined forcings are collected into a single dictionary, which is used to construct a hurricaneEvent.

In [ ]:
# Create the forcings dictionary
# The keys of the dictionary are the forcing types, and the values are lists of the corresponding forcing objects.
forcings = {
    f.ForcingType.WATERLEVEL: [water_levels],
    f.ForcingType.RAINFALL: [rainfall],
    f.ForcingType.WIND: [wind],
    f.ForcingType.DISCHARGE: [discharge1, discharge2],
}

# Create a hurricaneEvent with the forcings and time frame
event = HurricaneEvent(
    name="example_hurricane_event",
    time=time_frame,
    forcings=forcings,
    track_name=track.name,
    hurricane_translation=translation,
)

💾 7. Save the Event to a FloodAdapt Database

Finally, we save the event to a FloodAdapt database.

In [ ]:
# Save the event to the database
# fa = FloodAdapt(database_path=Path("path/to/database")) # TODO
# fa.save_event(event=event)